In [ ]:
#ссылка на датасет
original_url = 'https://www.kaggle.com/code/gcmadhan/world-happiness-index-report/input?select=world-happiness-report-2021.csv'
#тут получаю страны с разным индексом счастья
#мы предполагаем, что продавать успокаивающие чаи может быть идейно в страны с не очень большим индексом счастья (наш чай им обязательно поможет)
#потом из этих стран я выберу те страны, в которые реально доставлять наши чаи по их местоположению,
#так, чтобы можно было сделать в них доставку по суше и не усложнять перелетами и перевозками морем

import pandas as pd

df = pd.read_csv('/content/world-happiness-report-2021.csv')
df.sort_values('Healthy life expectancy')
good_columns = ['Country name', 'Regional indicator', 'Ladder score', 'Healthy life expectancy', ]
df = df[good_columns]
df

,Country name,Regional indicator,Ladder score,Healthy life expectancy
0,Finland,Western Europe,7.842,72.000
1,Denmark,Western Europe,7.620,72.700
2,Switzerland,Western Europe,7.571,74.400
3,Iceland,Western Europe,7.554,73.000
4,Netherlands,Western Europe,7.464,72.400
...,...,...,...,...
144,Lesotho,Sub-Saharan Africa,3.512,48.700
145,Botswana,Sub-Saharan Africa,3.467,59.269
146,Rwanda,Sub-Saharan Africa,3.415,61.400
147,Zimbabwe,Sub-Saharan Africa,3.145,56.201


In [ ]:
unique_regions = df['Regional indicator'].unique().tolist()
#удаляю регионы , куда мы при всем желании не доставим, что-то далеко и через море, много азии убираю, потому что они сами чаи много выращивают и зачем нам такая конкуренция
for index, row in df.iterrows():
  if row['Regional indicator'] in ['North America and ANZ', 'Middle East and North Africa', 'Latin America and Caribbean', 'East Asia', 'Sub-Saharan Africa', 'South Asia',  'Southeast Asia']:
    df.drop(index, inplace = True)
df

,Country name,Regional indicator,Ladder score,Healthy life expectancy
0,Finland,Western Europe,7.842,72.000
1,Denmark,Western Europe,7.620,72.700
2,Switzerland,Western Europe,7.571,74.400
3,Iceland,Western Europe,7.554,73.000
4,Netherlands,Western Europe,7.464,72.400
5,Norway,Western Europe,7.392,73.300
6,Sweden,Western Europe,7.363,72.700
7,Luxembourg,Western Europe,7.324,72.600
9,Austria,Western Europe,7.268,73.300
12,Germany,Western Europe,7.155,72.500


In [ ]:
url_general = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area'
#теперь получу ссылки на каждую страну и оттуда достану координаты (код частично взят из моей прошлой лабы)
#единственное, в моей лабе были не страны и была русскоязычная википедия, тут начальный датасет у меня на английском, поэтому опять беру англоязычную версию
import pandas as pd
import requests
from bs4 import BeautifulSoup

html = requests.get(url_general).text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', {'class': 'wikitable'})

#ссылочка в штуках с "а"
countries = []
links = []
for row in table.find_all('tr')[1:]:
    cells = row.find_all('td')

    country_cell = cells[1]
    country_name = country_cell.text.strip()
    link = country_cell.find('a')
    if link:
        link = "https://en.wikipedia.org" + link['href']
    else:
        link = None
    countries.append(country_name)
    links.append(link)

df_coord = pd.DataFrame({'Country name': countries, 'Link': links})
#удаляю первую строчку, там country name = earth, что не имеет смысла
df_coord = df_coord.drop(0)
#удаляю строку, где вместо страны было такое 9,525,067 (3,677,647)
df_coord = df_coord.drop(5)
#соединяю 2 датасета в один, чтобы потом найти координаты
df_coord = pd.merge(df, df_coord, on ='Country name')
df_coord

,Country name,Regional indicator,Ladder score,Healthy life expectancy,Link
0,Finland,Western Europe,7.842,72.000,https://en.wikipedia.org/wiki/Finland
1,Denmark,Western Europe,7.620,72.700,https://en.wikipedia.org/wiki/Denmark
2,Switzerland,Western Europe,7.571,74.400,https://en.wikipedia.org/wiki/Switzerland
3,Iceland,Western Europe,7.554,73.000,https://en.wikipedia.org/wiki/Iceland
4,Netherlands,Western Europe,7.464,72.400,https://en.wikipedia.org/wiki/Netherlands
5,Norway,Western Europe,7.392,73.300,https://en.wikipedia.org/wiki/Norway
6,Sweden,Western Europe,7.363,72.700,https://en.wikipedia.org/wiki/Sweden
7,Luxembourg,Western Europe,7.324,72.600,https://en.wikipedia.org/wiki/Luxembourg
8,Austria,Western Europe,7.268,73.300,https://en.wikipedia.org/wiki/Austria
9,Germany,Western Europe,7.155,72.500,https://en.wikipedia.org/wiki/Germany


In [ ]:
#беру свою функцию поиска координат из лабы 2 и меняю серединку, тк в англоязычной википедии немного другой формат данных
def get_coordinates(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        good_coord = soup.find_all('span', class_='geo')
        #в англоязычной википедии в начале страницы есть еще одни координаты, которые мне не нужны, поэтому беру второе вхождение
        good_coord =  good_coord[1].text
        lat = float(good_coord.split(';')[0].strip())
        lon = float(good_coord.split(';')[1].strip())
        return lat, lon
    except:
        return None, None

In [ ]:
df_coord['latitude'] = None
df_coord['longitude'] = None
for index, row in df_coord.iterrows():
    URL = row['Link']
    coordinates = get_coordinates(URL)
    df_coord.at[index, 'latitude'] = coordinates[0]
    df_coord.at[index, 'longitude'] = coordinates[1]
df_coord

,Country name,Regional indicator,Ladder score,Healthy life expectancy,Link,latitude,longitude
0,Finland,Western Europe,7.842,72.000,https://en.wikipedia.org/wiki/Finland,60.17083,24.9375
1,Denmark,Western Europe,7.620,72.700,https://en.wikipedia.org/wiki/Denmark,55.717,12.567
2,Switzerland,Western Europe,7.571,74.400,https://en.wikipedia.org/wiki/Switzerland,46.95,7.45
3,Iceland,Western Europe,7.554,73.000,https://en.wikipedia.org/wiki/Iceland,64.133,-21.933
4,Netherlands,Western Europe,7.464,72.400,https://en.wikipedia.org/wiki/Netherlands,52.367,4.883
5,Norway,Western Europe,7.392,73.300,https://en.wikipedia.org/wiki/Norway,59.933,10.683
6,Sweden,Western Europe,7.363,72.700,https://en.wikipedia.org/wiki/Sweden,59.35,18.067
7,Luxembourg,Western Europe,7.324,72.600,https://en.wikipedia.org/wiki/Luxembourg,49.6106,6.1328
8,Austria,Western Europe,7.268,73.300,https://en.wikipedia.org/wiki/Austria,48.2,16.35
9,Germany,Western Europe,7.155,72.500,https://en.wikipedia.org/wiki/Germany,52.517,13.383


In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

fig = px.choropleth(
    df_coord,
    locations='Country name',
    locationmode='country names',
    color='Ladder score',
    hover_name='Country name',
    hover_data='Healthy life expectancy',
    title='Уровень счастья по странам, в которые мы потенциально захотим продавать чаи',
    color_continuous_scale=px.colors.diverging.RdYlGn,
    projection='natural earth'
)
fig.show()